# 🔍 Python Embeddings через Cynosure Bridge

Полное руководство по работе с векторными embedding через OpenAI API совместимость.

## 🚀 Быстрый старт

In [ ]:
import openai
import numpy as np

# Подключение к Cynosure Bridge
client = openai.OpenAI(
    base_url="http://192.168.1.196:3000/v1",
    api_key="dummy-key"
)

# Создание embedding
response = client.embeddings.create(
    model="text-embedding-3-small",
    input="Пример текста для векторизации"
)

embedding = response.data[0].embedding
print(f"Размерность вектора: {len(embedding)}")
print(f"Использовано токенов: {response.usage.total_tokens}")

## 📊 Поддерживаемые модели

| OpenAI Model | Claude Alternative | Размерность | Применение |
|--------------|-------------------|-------------|------------|
| `text-embedding-3-small` | `claude-3-5-sonnet-20241022` | 1536 | Быстрая векторизация |
| `text-embedding-3-large` | `claude-3-5-sonnet-20241022` | 3072 | Высокое качество |
| `text-embedding-ada-002` | `claude-3-5-haiku-20241022` | 1536 | Совместимость |

## 💡 Семантический поиск

In [ ]:
def semantic_search(query, documents):
    # Векторизация запроса
    query_response = client.embeddings.create(
        model="text-embedding-3-small",
        input=query
    )
    query_embedding = query_response.data[0].embedding
    
    # Векторизация документов
    doc_embeddings = []
    for doc in documents:
        doc_response = client.embeddings.create(
            model="text-embedding-3-small",
            input=doc
        )
        doc_embeddings.append(doc_response.data[0].embedding)
    
    # Поиск наиболее похожего
    similarities = [
        np.dot(query_embedding, doc_emb) for doc_emb in doc_embeddings
    ]
    
    best_match_idx = np.argmax(similarities)
    return documents[best_match_idx], similarities[best_match_idx]

# Использование
documents = [
    "Python - язык программирования",
    "JavaScript используется для веб-разработки", 
    "Claude - это AI ассистент от Anthropic"
]

result, score = semantic_search("Что такое Python?", documents)
print(f"Найдено: {result} (score: {score:.3f})")

## 📦 Batch обработка

In [ ]:
def batch_embeddings(texts, batch_size=100):
    """Обработка больших массивов текста пакетами"""
    all_embeddings = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=batch
        )
        
        batch_embeddings = [item.embedding for item in response.data]
        all_embeddings.extend(batch_embeddings)
        
        print(f"Обработано {min(i + batch_size, len(texts))} из {len(texts)}")
    
    return all_embeddings

# Пример использования
large_text_list = [f"Текст номер {i}" for i in range(50)]
embeddings = batch_embeddings(large_text_list)
print(f"Получено {len(embeddings)} векторов")

## 🎯 Кластеризация документов

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

def cluster_documents(documents, n_clusters=3):
    # Получаем embeddings
    embeddings = []
    for doc in documents:
        response = client.embeddings.create(
            model="text-embedding-3-large",  # Высокое качество для кластеризации
            input=doc
        )
        embeddings.append(response.data[0].embedding)
    
    # Кластеризация
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(embeddings)
    
    return clusters, embeddings

# Применение
documents = [
    "Программирование на Python",
    "Веб-разработка с JavaScript", 
    "Машинное обучение с TensorFlow",
    "Создание API с FastAPI",
    "Frontend разработка с React",
    "Анализ данных с pandas"
]

clusters, embeddings = cluster_documents(documents)

for i, (doc, cluster) in enumerate(zip(documents, clusters)):
    print(f"Кластер {cluster}: {doc}")

## 🤖 RAG (Retrieval Augmented Generation)

In [ ]:
import faiss
import numpy as np

class SimpleRAG:
    def __init__(self):
        self.documents = []
        self.index = None
        
    def add_documents(self, docs):
        """Добавление документов в базу знаний"""
        self.documents.extend(docs)
        
        # Создаем embeddings
        embeddings = []
        for doc in docs:
            response = client.embeddings.create(
                model="text-embedding-3-small",
                input=doc
            )
            embeddings.append(response.data[0].embedding)
        
        # Создаем FAISS индекс
        dimension = len(embeddings[0])
        if self.index is None:
            self.index = faiss.IndexFlatIP(dimension)  # Inner Product
        
        embeddings_array = np.array(embeddings).astype('float32')
        self.index.add(embeddings_array)
    
    def search(self, query, k=3):
        """Поиск релевантных документов"""
        # Векторизация запроса
        query_response = client.embeddings.create(
            model="text-embedding-3-small",
            input=query
        )
        query_embedding = np.array([query_response.data[0].embedding]).astype('float32')
        
        # Поиск в индексе
        scores, indices = self.index.search(query_embedding, k)
        
        results = []
        for score, idx in zip(scores[0], indices[0]):
            if idx < len(self.documents):
                results.append((self.documents[idx], float(score)))
        
        return results
    
    def ask(self, question):
        """RAG: поиск + генерация ответа"""
        # Находим релевантные документы
        relevant_docs = self.search(question, k=2)
        
        # Формируем контекст
        context = "\n".join([doc for doc, _ in relevant_docs])
        
        # Генерируем ответ с контекстом
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": f"Отвечай на основе предоставленного контекста:\n{context}"},
                {"role": "user", "content": question}
            ]
        )
        
        return response.choices[0].message.content

# Использование RAG
rag = SimpleRAG()

# Добавляем документы в базу знаний
knowledge_base = [
    "Cynosure Bridge - это OpenAI-совместимый прокси для Claude",
    "Bridge работает на порту 3000 и поддерживает streaming",
    "Можно использовать любой OpenAI SDK, просто поменяв base URL",
    "Поддерживаются embeddings через /v1/embeddings endpoint"
]

rag.add_documents(knowledge_base)

# Задаем вопрос
answer = rag.ask("Как использовать Cynosure Bridge?")
print(answer)

## 🔧 Кэширование и оптимизация

In [ ]:
import pickle
import hashlib
import time

class EmbeddingCache:
    def __init__(self, cache_file="embeddings_cache.pkl"):
        self.cache_file = cache_file
        try:
            with open(cache_file, 'rb') as f:
                self.cache = pickle.load(f)
        except FileNotFoundError:
            self.cache = {}
    
    def get_embedding(self, text, model="text-embedding-3-small"):
        # Создаем ключ кэша
        key = hashlib.md5(f"{model}:{text}".encode()).hexdigest()
        
        if key in self.cache:
            print("Cache hit!")
            return self.cache[key]
        
        # Получаем embedding от API
        response = client.embeddings.create(model=model, input=text)
        embedding = response.data[0].embedding
        
        # Сохраняем в кэш
        self.cache[key] = embedding
        self.save_cache()
        
        return embedding
    
    def save_cache(self):
        with open(self.cache_file, 'wb') as f:
            pickle.dump(self.cache, f)

# Демонстрация кэширования
cache = EmbeddingCache()

# Первый вызов - будет запрос к API
start = time.time()
embedding1 = cache.get_embedding("Пример текста")
time1 = time.time() - start

# Второй вызов - из кэша
start = time.time()
embedding2 = cache.get_embedding("Пример текста")
time2 = time.time() - start

print(f"Первый запрос: {time1:.3f}с")
print(f"Второй запрос (кэш): {time2:.3f}с")
print(f"Ускорение: {time1/time2:.1f}x")